In [1]:
import os

import numpy as np
import pandas as pd
import networkx as nx

import arrow
from tqdm import tqdm

from dotenv import load_dotenv
from coinbase.wallet.client import Client

load_dotenv('.env')
client = Client(os.environ['COINBASE_KEY'], os.environ['COINBASE_SECRET'])

### Change here to select project output

In [10]:
# projects = ['bayc', 'coolcats', 'cryptoadz', 'cyberkongz', 'hashmasks', 'mayc', 'meebits', 'mekaverse', 'svs']
projects = ['bayc', 'coolcats']
# projects = ['coolcats', 'cryptoadz', 'cyberkongz', 'hashmasks', 'mayc', 'meebits', 'mekaverse', 'svs']

### Store base data as a dataframe

In [3]:
def create_base_data(project):
    PATH_TO_DATA = './data/collated/' + project + '.csv'  # Change if needed
    column_names = ["row", "tx_hash", "token_address", "from_address", "to_address", "token_id", "blk_number", "blk_timestamp", "eth_value"]
    
    df = pd.read_csv(PATH_TO_DATA, delimiter=',', skiprows=1, names=column_names)
    
    df["from_address"] = df.from_address.apply(lambda x: x.strip())
    df["to_address"] = df.to_address.apply(lambda x: x.strip())
    
    return df

### Transaction data

In [4]:
def get_transaction_data(project):
    PATH_TO_DATA = f"./data/balances/{project}.csv"
    return pd.read_csv(PATH_TO_DATA)

def lookup_transaction_value(df, block, account):
    value = 0
    
    if account == '0x0000000000000000000000000000000000000000':
        return value
    
    try:
        b = df[(df['block'] == block) & (df['address'] == account)]
        value = b['eth_value'].head(1).iat[0]
    except Exception as e:
        print(block,account)
    return value

# Convert ETH value to USD at specified date
prices = {}
def get_usd_value(date, eth_value):
    if eth_value == 0:
        return eth_value

    if date in prices:
        return float(prices[date]) * eth_value
    else:
        prices[date] = client.get_spot_price(currency_pair='ETH-USD', date=date)['amount']
        return float(prices[date]) * eth_value

### Build time-based dataframes

In [5]:
def create_timed_data(df, df_transactions):
    ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
    column_names = [
        "date", 
        "days_since_mint", 
        "from_address", 
        "to_address", 
        "token_id", 
        "blk_number", 
        "eth_value",
        #"usd_value",
        "from_value", 
        "to_value",
    ]
    
    df_time = pd.DataFrame(columns=column_names)
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        blk_timestamp = row['blk_timestamp']
        date = arrow.get(blk_timestamp).datetime

        from_address = row['from_address']
        to_address = row['to_address']
        token_id = row['token_id']
        blk_number = row['blk_number']
        eth_value = row['eth_value']
        
        if from_address == ZERO_ADDRESS:
            days_since_mint = 0
            mint_date = date
        else:
            days_since_mint = date - mint_date
            
        #usd_value = get_usd_value(date, eth_value)
            
        from_value = lookup_transaction_value(df_transactions, blk_number, from_address)
        to_value = lookup_transaction_value(df_transactions, blk_number, to_address)
            
        df_time = df_time.append({
            'date': date,
            'days_since_mint': days_since_mint,
            'from_address': from_address,
            'to_address': to_address,
            'token_id': token_id, 
            'blk_number': blk_number,
            'eth_value': eth_value,
            #'usd_value': usd_value,
            'from_value': from_value,
            'to_value': to_value,
        }, ignore_index=True)

    return df_time

### Build graph objects from time base dataframes

In [6]:
def build_graph_from_timed(df_time):    
    # Building a network per block
    # we will use a weighted and directed graph.
    graph = nx.MultiDiGraph()

    # loop over the pandas dataframe.
    for index, row in tqdm(df_time.iterrows(), total=df_time.shape[0]):
        # read the values from the dataframe.
        # token_id  blk_timestamp eth_value 
        date = row['date']
        from_address = row['from_address']
        to_address = row['to_address']
        token_id = row['token_id']
        blk_number = row['blk_number']
        eth_value = row['eth_value']
        #usd_value = row['usd_value']
        from_value = row['from_value']
        to_value = row['to_value']
        
        # make sure both addresses are in the graph.
        if from_address not in graph:
            graph.add_node(from_address)
        if to_address not in graph:
            graph.add_node(to_address)

        # set the attributes on this node.
        nx.set_node_attributes(graph, {from_address: from_value, to_address: to_value}, 'eth_value')

        # keep track of how many trades a wallet has done.
        trades = nx.get_node_attributes(graph, "trades")
        if from_address in trades:
            nx.set_node_attributes(graph, {from_address:trades[from_address] + 1}, 'trades')
        else:
            nx.set_node_attributes(graph, {from_address:1}, 'trades')
        if to_address in trades:
            nx.set_node_attributes(graph, {to_address:trades[to_address] + 1}, 'trades')
        else:
            nx.set_node_attributes(graph, {to_address:1}, 'trades')

        # check if this NFT has already been sold and if yes, remove the old sale.
        # this might be a candidate for memoization - c.b.
        remove_edges = []
        for (u,v,d) in graph.edges.data():
            if d['token_id'] == token_id:
                remove_edges.append((u,v))
        # we need to remove them in a seperate step, since otherwise we change the datastructure that we are iterating over.
        for (u,v) in remove_edges:
            graph.remove_edge(u,v)

        # add an edge for the transaction.
        graph.add_edge(from_address, to_address, weight=eth_value, token_id=token_id) # keep track of token id by adding it to the edge.
        
    return graph

### Build time-based snapshots

In [7]:
def build_graph_snapshots(df_time):
    """Simpler version of below function which just returns the graph objects"""
    res = {}
    
    df_time['date_quantile'], bins = pd.qcut(df_time['date'], 10, labels=False, retbins=True)
    time_buckets = np.unique(df_time["date_quantile"].to_numpy())
    
    for time_bucket, label in zip(time_buckets, bins):
        selection = df_time[(df_time['date_quantile'] == time_bucket)]
        graph_snapshot = build_graph_from_timed(selection)
        res[label] = graph_snapshot
        
    return res

In [8]:
def build_df_snapshots(df_time):
    column_names = [
        "time_bucket", 
        "time_bucket_label",
        "number_of_nodes", 
        "avg_clustering", 
        "reciprocity", 
        "assortativity", 
        "assortativity_base", 
        "assortativity_out_out", 
        "assortativity_in_in", 
        "assortativity_in_out",
        "centrality_degree",
        "centrality_closeness", 
        "centrality_betweenness",
        "centrality_eigenvector",
        "avg_clustering_random",
        "assortativity_random"
    ]
    
    df_snapshots = pd.DataFrame(columns=column_names)
    
    df_time['date_quantile'], bins = pd.qcut(df_time['date'], 10, labels=False, retbins=True)
    time_buckets = np.unique(df_time["date_quantile"].to_numpy())
    
    for time_bucket, label in zip(time_buckets, bins):
        selection = df_time[(df_time['date_quantile'] == time_bucket)]
        graph_snapshot = build_graph_from_timed(selection)
        
        df_snapshots = df_snapshots.append({
            "time_bucket": time_bucket,
            "time_bucket_label": label,
            "number_of_nodes": graph_snapshot.number_of_nodes(),
            "reciprocity": nx.reciprocity(graph_snapshot),
            "assortativity": nx.degree_assortativity_coefficient(graph_snapshot),
            "assortativity_base": nx.degree_pearson_correlation_coefficient(graph_snapshot.to_undirected(), weight='weight'),
            "assortativity_out_out": nx.degree_pearson_correlation_coefficient(graph_snapshot, x='out', y='out', weight='weight'),
            "assortativity_in_in": nx.degree_pearson_correlation_coefficient(graph_snapshot, x='in', y='in', weight='weight'),
            "assortativity_in_out": nx.degree_pearson_correlation_coefficient(graph_snapshot, x='in', y='out', weight='weight'),
            "centrality_degree": nx.degree_centrality(graph_snapshot),
            "centrality_closeness": nx.closeness_centrality(graph_snapshot),
        }, ignore_index=True)
        
    return df_snapshots.sort_values(by=['time_bucket'])

In [9]:
for project in projects:
    df_transactions = get_transaction_data(project)
    df_time = create_timed_data(create_base_data(project), df_transactions)
    g_time = build_graph_from_timed(df_time)
    
    np.save(f"./memory/{project}/full.npy", df_time)
    nx.write_gml(g_time, f"./memory/{project}/full.gml")

 99%|████████████████████████████████████▍| 44014/44619 [07:02<00:08, 74.80it/s]

13644678 0xceb78abfad166c11e350156fd671b385265a6c87
13644678 0xb3779bb48e7e580c00b4fc05dd4f7a8cd6b3b60a
13644797 0x64932f86d69f2717307f41b4c6b8198000583c63
13644797 0x61fcb701b0670d6534266cea68ed51ef127c0c1b
13644932 0xeee1b7b4ae1516d68d8c154efd803b4601a59be4
13644932 0x98f8cb751443cee2e0175a2072789e657317a404
13645134 0xa87bac952868726052dcce756c864f3bb2bbf4ff
13645134 0x4d2036407a023ea482db7ba19092d8ee08852ad1
13645489 0x21edc67e48471e4d3069603ee2fc15d823d59e2c
13645489 0xce9000536c859b6807c3986b03b73b54e8747cdf
13645535 0x21edc67e48471e4d3069603ee2fc15d823d59e2c
13645535 0xce9000536c859b6807c3986b03b73b54e8747cdf
13645923 0x7fa439f2257851e41b7ef6d78661df756ee37e3f
13645923 0xdc3e80869645fa33d89e1ecb87362e2d74f131c5
13645958 0x3458fd8f327f270b32092cd406ed8ba97405d9f5
13645958 0x200a69c1bfd00d1eb0d2c66eee07a49941b50b50
13646002 0x0e1a2155eb0ce4a58c2d242be0efea0b09e18d26
13646002 0x20d0a60b7446fb1154135e3b917a743fec0e5382
13646148 0x88341d1a8f672d2780c8dc725902aae72f143b0c
13646148 0xd

 99%|████████████████████████████████████▌| 44030/44619 [07:02<00:07, 75.30it/s]

 0xb2c5b6d70e2a1cd7c76ca3ee5213959a50348422
13647886 0xb697b9455eeee9574ec5c5350cb10c8a5b752a39
13647886 0x0e1f8708bfbfc3c9986bf6e393dd29d6885c66db
13648101 0x22164ed2db069056bceee58aaaa267c2a4aa51e7
13648101 0x2b84773955eee20e0ab23e44f0a174d92633fe84
13648138 0x33cac46e9a86d96ca0a88f076e281682ae714d02
13648138 0xe34906452d2268128f923fb1bf6ccfc4fd69b939
13648452 0x0cef74d157dedba45d59d9f6c8232ce49bba0bff
13648452 0x1fc78da4989ef654ca2dadae88b27bb0e6f8d9c4
13648556 0xf1637adedeb89559c70481c3cb4f74ebac80d829
13648556 0xf224837e13b1a745574ef68a73427b0063f266f1
13648563 0x9edb4bf34266684e42a17a5db8099965218d4d7d
13648563 0xf750b3ba78b0ea76e6600eba37bc6d84fe87734e
13648584 0xdb24f4bdf3f171f8f691cf9d2d4cb4e77675453b
13648584 0x4748495153fb86637e4fdd8e50e3c1f611f15930
13648604 0x4748495153fb86637e4fdd8e50e3c1f611f15930
13648604 0xa8362ae224258d89c5d04e13829062a5c4121d0f
13648776 0xf3137fea934af1e6764aee22c5eaf6a2887cd76a
13648776 0x72a53ed98a7bca05eda8ceb1b13bdc9490ff402c
13648828 0x342850f5a

 99%|████████████████████████████████████▌| 44047/44619 [07:03<00:07, 78.42it/s]

...

 0x1e23166b3b73dd2ee9d9db190aceeee8f8edd0aa
13725343 0x070d0bd96790fb1fb9fa0df5890c2e40e02e32ee
13725343 0x407ed94415b5ed61d1f30950520d658a59a21ed3
13725878 0xce4c497b87e239fc20385f759fa69fab3abcb37b
13725878 0x5d7dcb9f59d4e1cf96463a72e866966149df1552
13726020 0x05428e2ae62cfcb7b168f3152194305a7068fb11
13726020 0xd6dbf4e7e92b8fc1569ec79c48ff9f6fdf997d35
13726120 0xa3599cf1048930c9e897fe9245dd95511cd16f1c
13726120 0x006ebf6895474a1461f05233d23d1eadaf8c4c2e
13726152 0x12d4c030d37885375ec42753b1923b2a866cc31d
13726152 0xd5db7354a34a265257e957b31b45f49c3943f1b2
13726654 0x88341d1a8f672d2780c8dc725902aae72f143b0c
13726654 0x9002b80e3fd2cfe9bb04d89438c55a0d08dd218c
13726835 0x261d4ce70108035f82fe07a8d90a413476417da1
13726835 0x35389efc9b5e87748c8e2a86dc4e754ea2cd8f03
13727368 0xc580950681be90e085734509b8cdcb6b16ad41b3
13727368 0xa072e54e1c81d4f930048e1c3f43d506d396ad97
13727368 0xc580950681be90e085734509b8cdcb6b16ad41b3
13727368 0xa072e54e1c81d4f930048e1c3f43d506d396ad97
13727464 0xe0404bb00

100%|████████████████████████████████████| 44619/44619 [07:10<00:00, 103.54it/s]


13729062 0x6a85d8cc3c67d9c41c594162779074e0bf9a74e1
13729062 0xf224837e13b1a745574ef68a73427b0063f266f1
13729213 0x34e923a41b714a79effa74b745fa72a94496a5ef
13729213 0xae741385b1959bcb815821d99dcefedfe8120f78
13729263 0xe971e45db57eb8ca8aa2bb2dc6604afd5ea1a1d6
13729263 0xbb0d4ccf4e095a2d6a9a2bee2985a703c1ca9b69
13729282 0xa9e600444278aab04eaa9655ac94478bf7c4e2e7
13729282 0x5e7e4ea979b371f5f23191cb292237d60c41c1ff
13729292 0x5e7e4ea979b371f5f23191cb292237d60c41c1ff
13729292 0xa9e600444278aab04eaa9655ac94478bf7c4e2e7
13729952 0xea70f5fcf0e51900d13b7992b0996530f036a133
13729952 0xea70f5fcf0e51900d13b7992b0996530f036a133
13730008 0xbbe02c02489143c77a2e1b06d9f75726061b296b
13730008 0x38d2276032593c7cc6d0ec58aff942c0fd706ce1
13730008 0x3ba312d751874004036db73145ca9ccbb1a84824
13730008 0x38d2276032593c7cc6d0ec58aff942c0fd706ce1
13730008 0xf7e20ed715a0d83f89130796c44c3d36c724a0c4
13730008 0x38d2276032593c7cc6d0ec58aff942c0fd706ce1
13730008 0xe07ba08531027f71000c917b5617c00669725821
13730008 0x3

100%|████████████████████████████████████| 44619/44619 [03:38<00:00, 204.24it/s]


FileNotFoundError: [Errno 2] No such file or directory: './data/balances/cryptoadz.csv'

In [11]:
for project in tqdm(projects):
    column_names = [
        "date", 
        "days_since_mint", 
        "from_address", 
        "to_address", 
        "token_id", 
        "blk_number", 
        "eth_value",
        "from_value", 
        "to_value",
    ]
    
    np_data = np.load(f"./memory/{project}/full.npy", allow_pickle=True)
    df_time = pd.DataFrame(data=np_data, columns=column_names)
    
    g_snapshots = build_graph_snapshots(df_time)
    df_snapshot_summary = build_df_snapshots(df_time)
    
    for i, snapshot in enumerate(g_snapshots.keys()):
        nx.write_gml(g_time, f"./memory/{project}/snapshots/{i}.gml")
    
    np.save(f"./memory/{project}/snapshots/summary.npy", df_time)

100%|█████████████████████████████████████| 5099/5099 [00:03<00:00, 1312.68it/s]

100%|█████████████████████████████████████| 5126/5126 [00:03<00:00, 1463.43it/s]

100%|██████████████████████████████████████| 5060/5060 [00:06<00:00, 789.29it/s]

100%|██████████████████████████████████████| 5095/5095 [00:07<00:00, 727.05it/s]

 56%|█████████████████████▎                | 2858/5095 [00:02<00:02, 750.43it/s]


100%|██████████████████████████████████████| 5095/5095 [00:06<00:00, 778.47it/s]

100%|██████████████████████████████████████| 5095/5095 [00:06<00:00, 784.50it/s]

100%|██████████████████████████████████████| 5096/5096 [00:07<00:00, 707.19it/s]

 73%|███████████████████████████▊          | 3725/5094 [00:04<00:02, 518.70it/s]


100%|██████████████████████████████████████| 5094/5094 [00:07<00:00, 692.45it/s]

100%|██████████████████████████████████████| 5095/5095 [00:08<00:00, 634.07it/s]

100%|██████████████████████████████████████| 5095/5095 [00:07<00:00, 677.20it/s]

 95%|████████████████████████████████████  | 4837/5099 [00:03<00:00, 800.84it/s]


100%|█████████████████████████████████████| 5099/5099 [00:03<00:00, 1303.05it/s]

100%|█████████████████████████████████████| 5126/5126 [00:03<00:00, 1485.07it/s]

100%|██████████████████████████████████████| 5060/5060 [00:06<00:00, 785.47it/s]

100%|██████████████████████████████████████| 5095/5095 [00:07<00:00, 683.37it/s]

 88%|█████████████████████████████████▍    | 4485/5095 [00:05<00:01, 510.58it/s]


100%|██████████████████████████████████████| 5095/5095 [00:06<00:00, 782.54it/s]

100%|██████████████████████████████████████| 5095/5095 [00:06<00:00, 797.00it/s]

100%|██████████████████████████████████████| 5096/5096 [00:07<00:00, 709.46it/s]

 98%|█████████████████████████████████████▎| 4994/5094 [00:07<00:00, 394.64it/s]


100%|██████████████████████████████████████| 5094/5094 [00:07<00:00, 690.58it/s]

100%|██████████████████████████████████████| 5095/5095 [00:07<00:00, 647.80it/s]

100%|█████████████████████████████████████| 4462/4462 [00:02<00:00, 1495.95it/s]

100%|█████████████████████████████████████| 4509/4509 [00:02<00:00, 1628.71it/s]

 81%|██████████████████████████████▋       | 3569/4415 [00:02<00:01, 820.36it/s]


100%|█████████████████████████████████████| 4415/4415 [00:03<00:00, 1177.34it/s]

100%|██████████████████████████████████████| 4463/4463 [00:04<00:00, 949.26it/s]

100%|██████████████████████████████████████| 4461/4461 [00:05<00:00, 875.05it/s]

100%|██████████████████████████████████████| 4461/4461 [00:05<00:00, 794.52it/s]

100%|██████████████████████████████████████| 4462/4462 [00:06<00:00, 725.93it/s]

  0%|                                                  | 0/4462 [00:00<?, ?it/s]


100%|██████████████████████████████████████| 4462/4462 [00:06<00:00, 700.21it/s]

100%|██████████████████████████████████████| 4462/4462 [00:06<00:00, 716.42it/s]

100%|██████████████████████████████████████| 4462/4462 [00:05<00:00, 759.82it/s]

100%|█████████████████████████████████████| 4462/4462 [00:02<00:00, 1502.06it/s]

 88%|████████████████████████████████▋    | 3990/4509 [00:02<00:00, 1114.89it/s]


100%|█████████████████████████████████████| 4509/4509 [00:02<00:00, 1644.86it/s]

100%|█████████████████████████████████████| 4415/4415 [00:03<00:00, 1188.57it/s]

100%|██████████████████████████████████████| 4463/4463 [00:04<00:00, 959.45it/s]

100%|██████████████████████████████████████| 4461/4461 [00:05<00:00, 884.88it/s]

100%|██████████████████████████████████████| 4461/4461 [00:05<00:00, 803.78it/s]

 79%|█████████████████████████████▉        | 3518/4462 [00:03<00:01, 516.19it/s]


100%|██████████████████████████████████████| 4462/4462 [00:06<00:00, 740.09it/s]

100%|██████████████████████████████████████| 4462/4462 [00:06<00:00, 689.54it/s]

100%|██████████████████████████████████████| 4462/4462 [00:06<00:00, 716.18it/s]

100%|████████████████████████████████████████████| 2/2 [04:05<00:00, 122.66s/it]
